## Home Credit Default Risk Prediction

### Project Overview

This project aims to develop multiple machine learning models to predict the likelihood of loan default for Home Credit Group clients. As a proof-of-concept (POC) for our startup's risk evaluation service, we'll create a suite of models that can help banks make more informed decisions about loan approvals. Our goal is to maximize accuracy in identifying both creditworthy clients and potential defaults, providing a robust and flexible solution for our potential banking clients.

### Why is this Important?

Home Credit specializes in lending to individuals with limited or no credit history (often referred to as "unbanked" or "underbanked"). They utilize diverse data sources, including phone and transaction data, to assess loan repayment probability. By developing accurate default prediction models, we can:

- **Expand financial inclusion:** Enable deserving clients who might be rejected by traditional methods to access credit.
- **Optimize risk management:** Help financial institutions make more informed lending decisions, reducing their exposure to potential losses.
- **Demonstrate our startup's capabilities:** Showcase our ability to translate complex business requirements into effective machine learning solutions.

### Dataset Description

We're using the Home Credit Default Risk dataset, which includes comprehensive information about loan applications and borrowers' credit histories. The data is structured across several interconnected tables:

1. **Application Data (`application_{train|test}.csv`):** The primary table containing loan application details such as age, income, employment history, etc.
2. **Bureau Data (`bureau.csv` and `bureau_balance.csv`):** Information about the applicants' previous loans from other financial institutions.
3. **Previous Applications (`previous_application.csv`):** Historical data on the applicants' past loan applications with Home Credit.
4. **Home Credit Loan Details:**
   - **POS and Cash Loans Balance (`pos_cash_balance.csv`):** Monthly balance snapshots of previous point-of-sale and cash loans.
   - **Credit Card Balance (`credit_card_balance.csv`):** Monthly balance snapshots of previous credit cards.
5. **Installments Payments (`installments_payments.csv`):** Payment history for previous loans at Home Credit.

### Our Approach

We'll use a comprehensive, multi-stage approach to develop our predictive models:

1. **Initial Data Exploration:**

   - Conduct thorough Exploratory Data Analysis (EDA) on each table.
   - Identify key variables and their distributions.
   - Investigate relationships between features and the target variable (loan default).
   - Check for data quality issues, missing values, and anomalies.

2. **Feature Engineering and Data Preprocessing:**

   - Create new features based on domain knowledge and initial insights.
   - Handle missing data and outliers.
   - Perform appropriate encoding for categorical variables and scaling for numerical features.

3. **Statistical Inference:**

   - Define the target population and formulate multiple statistical hypotheses.
   - Construct confidence intervals and conduct appropriate statistical tests.
   - Analyze correlations and other relationships between variables.

4. **Model Development:**

   - Implement multiple machine learning algorithms (e.g., Logistic Regression, Random Forests, Gradient Boosting).
   - Utilize cross-validation techniques to ensure model robustness.
   - Perform hyperparameter tuning to optimize model performance.

5. **Model Evaluation and Selection:**

   - Assess models using appropriate performance metrics (e.g., AUC-ROC, precision, recall).
   - Analyze feature importance and model interpretability.
   - Select the best-performing models for deployment.

6. **Model Deployment:**

   - Deploy the top-performing models to Google Cloud Platform.
   - Ensure models are accessible via HTTP requests for easy integration.

7. **Documentation and Presentation:**
   - Clearly document all steps, assumptions, and results.
   - Prepare visualizations and explanations of our findings.
   - Develop recommendations for potential clients based on our insights.

### Additional Notes

- **Data Source:** The dataset is from the [Home Credit Default Risk Kaggle Competition](https://www.kaggle.com/competitions/home-credit-default-risk/data).
- **Geographic Scope:** Home Credit operates primarily in CIS and Southeast Asian countries, including Kazakhstan, Russia, Vietnam, China, Indonesia, and the Philippines.
- **Product Categories:** Home Credit offers various loan products:
  - Revolving loans (credit cards)
  - Consumer loans (point-of-sale or POS loans)
  - Cash loans
- **Ethical Considerations:** We'll pay close attention to potential biases in our models and strive for fair lending practices.
- **Iterative Process:** Our approach will be flexible, allowing for iterations and refinements based on insights gained throughout the project.

**Home Credit Dataset Schema:**

![Home Credit Dataset Schema](../images/data-scheme.png)

This project will demonstrate our ability to handle complex, real-world data and deliver valuable insights and predictive capabilities to the financial sector.

By the end of this project, we aim to provide Home Credit with a reliable and easy-to-use tool to predict loan default risk, allowing them to make better lending decisions and potentially extend credit to more people who can manage it responsibly.


In [49]:
from typing import Dict
import pandas as pd
import polars as pl
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
import xgboost as xgb
import lightgbm as lgb
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.pyplot as plt

from retail_bank_risk.data_preprocessing_utils import (
    handle_missing_values,
    simple_imputation,
    flag_anomalies,
    reduce_memory_usage_pl,
    initial_feature_reduction,
    get_top_missing_value_percentages,
    analyze_missing_values

)
from retail_bank_risk.basic_visualizations_utils import (
    plot_correlation_matrix,
    plot_combined_histograms,
    plot_categorical_features_by_target,
    plot_combined_bar_charts,
    plot_combined_boxplots,
    plot_single_bar_chart,
    plot_feature_importances,
)
from retail_bank_risk.model_utils import (
    evaluate_model,
    extract_feature_importances,
)
from retail_bank_risk.advanced_visualizations_utils import (
    plot_model_performance,
    plot_combined_confusion_matrices,
    plot_roc_curve,
    plot_precision_recall_curve,
    plot_confusion_matrix,
    plot_learning_curve,
)

For reproducibility, we set a random seed.


In [4]:
np.random.seed(42)

We then load the CSV files using Polars.


In [5]:
application_train = pl.read_csv("../data/application_train.csv")
application_test = pl.read_csv("../data/application_test.csv")
previous_application = pl.read_csv("../data/previous_application.csv")
pos_cash_balance = pl.read_csv("../data/pos_cash_balance.csv")
installments_payments = pl.read_csv("../data/installments_payments.csv")
bureau = pl.read_csv("../data/bureau.csv")
bureau_balance = pl.read_csv("../data/bureau_balance.csv")
credit_card_balance = pl.read_csv("../data/credit_card_balance.csv")

We've loaded all the data tables, but our initial focus will be on the application_train DataFrame.

This table contains the core details of each loan application, which might be sufficient for building our first models.

If we need more information to improve accuracy, we'll then explore the other tables and incorporate relevant data from them.

Next up we will check the first rows of our train and test datasets.


In [6]:
application_train.head()

SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,…,NONLIVINGAREA_MEDI,FONDKAPREMONT_MODE,HOUSETYPE_MODE,TOTALAREA_MODE,WALLSMATERIAL_MODE,EMERGENCYSTATE_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
i64,i64,str,str,str,str,i64,f64,f64,f64,f64,str,str,str,str,str,f64,i64,i64,f64,i64,f64,i64,i64,i64,i64,i64,i64,str,f64,i64,i64,str,i64,i64,i64,i64,…,f64,str,str,f64,str,str,f64,f64,f64,f64,f64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,f64,f64,f64,f64,f64,f64
100002,1,"""Cash loans""","""M""","""N""","""Y""",0,202500.0,406597.5,24700.5,351000.0,"""Unaccompanied""","""Working""","""Secondary / secondary special""","""Single / not married""","""House / apartment""",0.018801,-9461,-637,-3648.0,-2120,null,1,1,0,1,1,0,"""Laborers""",1.0,2,2,"""WEDNESDAY""",10,0,0,0,…,0.0,"""reg oper account""","""block of flats""",0.0149,"""Stone, brick""","""No""",2.0,2.0,2.0,2.0,-1134.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
100003,0,"""Cash loans""","""F""","""N""","""N""",0,270000.0,1293502.5,35698.5,1.1295e6,"""Family""","""State servant""","""Higher education""","""Married""","""House / apartment""",0.003541,-16765,-1188,-1186.0,-291,null,1,1,0,1,1,0,"""Core staff""",2.0,1,1,"""MONDAY""",11,0,0,0,…,0.01,"""reg oper account""","""block of flats""",0.0714,"""Block""","""No""",1.0,0.0,1.0,0.0,-828.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
100004,0,"""Revolving loans""","""M""","""Y""","""Y""",0,67500.0,135000.0,6750.0,135000.0,"""Unaccompanied""","""Working""","""Secondary / secondary special""","""Single / not married""","""House / apartment""",0.010032,-19046,-225,-4260.0,-2531,26.0,1,1,1,1,1,0,"""Laborers""",1.0,2,2,"""MONDAY""",9,0,0,0,…,null,null,null,null,null,null,0.0,0.0,0.0,0.0,-815.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
100006,0,"""Cash loans""","""F""","""N""","""Y""",0,135000.0,312682.5,29686.5,297000.0,"""Unaccompanied""","""Working""","""Secondary / secondary special""","""Civil marriage""","""House / apartment""",0.008019,-19005,-3039,-9833.0,-2437,null,1,1,0,1,0,0,"""Laborers""",2.0,2,2,"""WEDNESDAY""",17,0,0,0,…,null,null,null,null,null,null,2.0,0.0,2.0,0.0,-617.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,null,null,null,null,null,null
100007,0,"""Cash loans""","""M""","""N""","""Y""",0,121500.0,513000.0,21865.5,513000.0,"""Unaccompanied""","""Working""","""Secondary / secondary special""","""Single / not married""","""House / apartment""",0.028663,-19932,-3038,-4311.0,-3458,null,1,1,0,1,0,0,"""Core staff""",1.0,2,2,"""THURSDAY""",11,0,0,0,…,null,null,null,null,null,null,0.0,0.0,0.0,0.0,-1106.0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0


In [7]:
application_test.head()

SK_ID_CURR,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,…,NONLIVINGAREA_MEDI,FONDKAPREMONT_MODE,HOUSETYPE_MODE,TOTALAREA_MODE,WALLSMATERIAL_MODE,EMERGENCYSTATE_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
i64,str,str,str,str,i64,f64,f64,f64,f64,str,str,str,str,str,f64,i64,i64,f64,i64,f64,i64,i64,i64,i64,i64,i64,str,f64,i64,i64,str,i64,i64,i64,i64,i64,…,f64,str,str,f64,str,str,f64,f64,f64,f64,f64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,f64,f64,f64,f64,f64,f64
100001,"""Cash loans""","""F""","""N""","""Y""",0,135000.0,568800.0,20560.5,450000.0,"""Unaccompanied""","""Working""","""Higher education""","""Married""","""House / apartment""",0.01885,-19241,-2329,-5170.0,-812,null,1,1,0,1,0,1,null,2.0,2,2,"""TUESDAY""",18,0,0,0,0,…,null,null,"""block of flats""",0.0392,"""Stone, brick""","""No""",0.0,0.0,0.0,0.0,-1740.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
100005,"""Cash loans""","""M""","""N""","""Y""",0,99000.0,222768.0,17370.0,180000.0,"""Unaccompanied""","""Working""","""Secondary / secondary special""","""Married""","""House / apartment""",0.035792,-18064,-4469,-9118.0,-1623,null,1,1,0,1,0,0,"""Low-skill Laborers""",2.0,2,2,"""FRIDAY""",9,0,0,0,0,…,null,null,null,null,null,null,0.0,0.0,0.0,0.0,0.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,3.0
100013,"""Cash loans""","""M""","""Y""","""Y""",0,202500.0,663264.0,69777.0,630000.0,null,"""Working""","""Higher education""","""Married""","""House / apartment""",0.019101,-20038,-4458,-2175.0,-3503,5.0,1,1,0,1,0,0,"""Drivers""",2.0,2,2,"""MONDAY""",14,0,0,0,0,…,null,null,null,null,null,null,0.0,0.0,0.0,0.0,-856.0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,1.0,4.0
100028,"""Cash loans""","""F""","""N""","""Y""",2,315000.0,1.575e6,49018.5,1.575e6,"""Unaccompanied""","""Working""","""Secondary / secondary special""","""Married""","""House / apartment""",0.026392,-13976,-1866,-2000.0,-4208,null,1,1,0,1,1,0,"""Sales staff""",4.0,2,2,"""WEDNESDAY""",11,0,0,0,0,…,0.0817,"""reg oper account""","""block of flats""",0.37,"""Panel""","""No""",0.0,0.0,0.0,0.0,-1805.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,3.0
100038,"""Cash loans""","""M""","""Y""","""N""",1,180000.0,625500.0,32067.0,625500.0,"""Unaccompanied""","""Working""","""Secondary / secondary special""","""Married""","""House / apartment""",0.010032,-13040,-2191,-4000.0,-4262,16.0,1,1,1,1,0,0,null,3.0,2,2,"""FRIDAY""",5,0,0,0,0,…,null,null,null,null,null,null,0.0,0.0,0.0,0.0,-821.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,null,null,null,null,null,null


We can see that there are many features. As expected, the test dataset does not include the target variable.

Lets check the exact number of rows and features we do have here.


In [8]:
print("Number of Rows and Columns in Application Train:", application_train.shape)
print("Number of Rows and Columns in Application Test:", application_test.shape)

Number of Rows and Columns in Application Train: (307511, 122)
Number of Rows and Columns in Application Test: (48744, 121)


Next up, before proceeding, we will reduce the dataset's memory footprint by converting the data types to lower-precision alternatives. This will lead to faster processing times.


In [9]:
application_train = reduce_memory_usage_pl(application_train)
application_test = reduce_memory_usage_pl(application_test)

Size before memory reduction: 289.68 MB
Initial data types: Counter({Float64: 65, Int64: 41, String: 16})
Size after memory reduction: 113.59 MB
Final data types: Counter({Float32: 65, Int8: 35, Categorical(ordering='physical'): 16, Int32: 4, Int16: 2})
Size before memory reduction: 45.51 MB
Initial data types: Counter({Float64: 65, Int64: 40, String: 16})
Size after memory reduction: 17.95 MB
Final data types: Counter({Float32: 65, Int8: 34, Categorical(ordering='physical'): 16, Int32: 4, Int16: 2})


To improve our training and testing datasets, we will remove features based on missing values and variance. Features with a missing value ratio above `missing_threshold` (50%) or a variance below `variance_threshold` will be removed.

Columns with a variance less than or equal to 0.01 are considered to have insufficient variability and are removed from the dataset. This is because a variance of 0.01 implies very little change in the values of that feature, making it potentially less useful for distinguishing between different observations in predictive modeling tasks.

This process will improve data quality, reduce dimensionality, and improve the performance of the credit risk model.

In [10]:
target_column = "TARGET"
reduced_train, reduced_test = initial_feature_reduction(
    application_train, application_test, target_column
)

print(f"Original number of features in train: {application_train.shape[1]}")
print(f"Number of features in reduced train: {reduced_train.shape[1]}")
print(f"Original number of features in test: {application_test.shape[1]}")
print(f"Number of features in reduced test: {reduced_test.shape[1]}")

Original number of features in train: 122
Number of features in reduced train: 85
Original number of features in test: 121
Number of features in reduced test: 84


sys:1: CategoricalRemappingWarning: Local categoricals have different encodings, expensive re-encoding is done to perform this merge operation. Consider using a StringCache or an Enum type if the categories are known in advance


In [70]:
analyze_missing_values(reduced_train, reduced_test)

Top 5 columns with missing values in reduced train set:
shape: (5, 2)
┌─────────────────┬────────────────────┐
│ column          ┆ missing_percentage │
│ ---             ┆ ---                │
│ str             ┆ f64                │
╞═════════════════╪════════════════════╡
│ LIVINGAREA_AVG  ┆ 50.19              │
│ LIVINGAREA_MODE ┆ 50.19              │
│ LIVINGAREA_MEDI ┆ 50.19              │
│ HOUSETYPE_MODE  ┆ 50.18              │
│ FLOORSMAX_AVG   ┆ 49.76              │
└─────────────────┴────────────────────┘

Top 5 columns with missing values in reduced test set:
shape: (5, 2)
┌─────────────────┬────────────────────┐
│ column          ┆ missing_percentage │
│ ---             ┆ ---                │
│ str             ┆ f64                │
╞═════════════════╪════════════════════╡
│ HOUSETYPE_MODE  ┆ 48.46              │
│ LIVINGAREA_AVG  ┆ 48.32              │
│ LIVINGAREA_MODE ┆ 48.32              │
│ LIVINGAREA_MEDI ┆ 48.32              │
│ FLOORSMAX_AVG   ┆ 47.84              │

**application_train:** 307,511 rows, 122 columns

**application_test:** 48,744 rows, 121 columns (excluding the TARGET variable)

We have a large training dataset with over **300k loan applications**. This provides a solid foundation for building robust machine learning models. The test dataset is well-sized for evaluating model performance on unseen data.

Both datasets have many features (**over 120**). This allows us to capture complex relationships and build accurate predictive models. However, the high number of features means we need to be careful in our selection and engineering process. This will help us avoid overfitting and keep our models interpretable.

Therefore, we will **manually select features based on their descriptions in the application_train and application_test DataFrames.**

1. **TARGET** (for training set only): Our dependent variable, shows if a client had payment difficulties.

2. **AMT_INCOME_TOTAL:** Income is key in determining loan repayment ability.

3. **AMT_CREDIT:** The loan amount directly relates to default risk.

4. **AMT_ANNUITY:** Annual loan payment affects repayment ability.

5. **NAME_CONTRACT_TYPE:** Different loan types may have varying risk profiles.

6. **CODE_GENDER:** Gender might influence repayment behavior.

7. **FLAG_OWN_CAR and FLAG_OWN_REALTY**: Asset ownership can indicate financial stability and potential collateral.

8. **CNT_CHILDREN:** Number of dependents affects financial obligations.

9. **NAME_EDUCATION_TYPE:** Education level often relates to income stability and financial literacy.

10. **DAYS_BIRTH:** Age is important in credit risk assessment.

11. **DAYS_EMPLOYED:** Employment duration can show job stability.

12. **NAME_INCOME_TYPE:** Income source affects its stability and reliability.

13. **NAME_FAMILY_STATUS:** Family status can impact financial responsibilities and stability.

14. **REGION_POPULATION_RELATIVE:** Economic opportunities may vary by region population.

15. **DAYS_LAST_PHONE_CHANGE:** Recent changes might suggest instability.

16. **EXT_SOURCE_1, EXT_SOURCE_2, EXT_SOURCE_3:** These scores likely include credit scores or similar risk assessments.

17. **OCCUPATION_TYPE:** Different jobs have varying levels of security and income stability.

18. **AMT_GOODS_PRICE:** For consumer loans, this shows the value of financed goods.

19. **OBS_30_CNT_SOCIAL_CIRCLE and DEF_30_CNT_SOCIAL_CIRCLE:** Financial behavior of one's social circle can indicate personal financial behavior.

20. **AMT_REQ_CREDIT_BUREAU_YEAR:** Number of credit inquiries in the past year can show recent financial activity or distress.

These features cover various aspects of a loan applicant's profile, including financial capacity, demographics, assets, employment, credit history, and behavior. This selection aims to provide a comprehensive view for assessing credit risk.


In [ ]:
target_column = "TARGET"
reduced_train, reduced_test = initial_feature_reduction(
    application_train, application_test, target_column
)

print(f"Original number of features in train: {application_train.shape[1]}")
print(f"Number of features in reduced train: {reduced_train.shape[1]}")
print(f"Original number of features in test: {application_test.shape[1]}")
print(f"Number of features in reduced test: {reduced_test.shape[1]}")

Original number of features in train: 122
Number of features in reduced train: 85
Original number of features in test: 121
Number of features in reduced test: 84


sys:1: CategoricalRemappingWarning: Local categoricals have different encodings, expensive re-encoding is done to perform this merge operation. Consider using a StringCache or an Enum type if the categories are known in advance
